In [17]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd

In [18]:
def busca_impi(medicamento):
    #Función para extraer los elementos de la tabla
    def elementos_tabla(driver):
        tabla=driver.find_element(By.CLASS_NAME, "ms-listviewtable")
        tabla_content=tabla.get_attribute('outerHTML')
        return tabla_content
        #Código para extraer los datos de la página
    url = "https://patmedsp.impi.gob.mx/Lists/MedicamentosPat/MIO.aspx"
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-cache')
    options.add_argument('--disable-cookies')
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service("C:/chromedriver.exe"), options=options)
    driver.get(url)
    time.sleep(5)
    cuadro_busqueda = driver.find_element(By.ID, "inplaceSearchDiv_WPQ1_lsinput")
    cuadro_busqueda.send_keys(medicamento)
    #Dar enter
    cuadro_busqueda.send_keys(u'\ue007')
    #Obtener el html
    time.sleep(5)
    try:
        table_contents=elementos_tabla(driver)
    except NoSuchElementException:
        driver.quit()
        return pd.DataFrame()
    
    #Obtener el html
    soup = BeautifulSoup(table_contents, 'html.parser')
    renglones = soup.find_all('tr')
    # #Obtener columnas de renglones
    columnas = [tr.find_all('td') for tr in renglones]
    #Dejar solo texto de columnas
    columnas = [[ele.text.strip() for ele in col] for col in columnas]
    
    #Crear dataframe
    df = pd.DataFrame(columnas)
    if df.empty:
        driver.quit()
        #Regresar dataframe con columnas vacías
        return pd.DataFrame()
    
    #Limpieza de dataframe
    df = df.iloc[1:]
    #Eliminar primera columna
    df = df.iloc[:,1:]
    #Renombrar columnas
    df.columns = ['id_ficha', 'num_concesion', 'titular', 'nom_generico', 'desc_esp', 'vigencia', 'anualidades', 'reivindicacines', 'obs', 
              'tipo_patente', 'enlace_med', 'ano_vigente', 'ind_terap', 'naturaleza','modif','creado']
    #Eliminar espacios en blanco de todas las columnas
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    #Eliminar \n de todas las columnas
    df = df.apply(lambda x: x.str.replace('\n', ' ') if x.dtype == "object" else x)
    #Minúsculas todas las columnas
    df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
    #Eliminar ñ y acentos de todas las columnas
    df = df.apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') if x.dtype == "object" else x)
    driver.quit()
    return df    

In [20]:
busca_impi("imatinib")

,id_ficha,num_concesion,titular,nom_generico,desc_esp,vigencia,anualidades,reivindicacines,obs,tipo_patente,enlace_med,ano_vigente,ind_terap,naturaleza,modif,creado
1,15,252475,novartis ag,imatinib,mesilato de imatinib,22-abril-2023,pago cubierto hasta el final de la vigencia,reivindicacion 1. una tableta que comprende un...,tipo de patente: composicion farmaceutica l...,composicion farmaceutica,mx/2008/28739,2023,"cancer, desordenes proliferativos no malignos ...",,22 de febrero,mario iran florentino parra hernandez
2,,270484,elan pharma international ltd.,mesilato de imatinib,,05-junio-2026,"ultimo pago: 01 octubre de 2009, siguiente pag...",,,s/i,mx/t/2009/85610,2026,,,23 de febrero,karina aurora gonzalez segura
3,,275628,"xenoport, inc.",mesilato de imatinib (forma ),,04-diciembre-2026,"ultimo pago: 25 noviembre de 2015, siguiente p...",,,s/i,mx/t/2010/35353,2026,,,23 de febrero,karina aurora gonzalez segura
4,,279837,novartis ag.,mesilato de imatinib (forma ),,02-junio-2026,"ultimo pago: 08 octubre de 2010, siguiente pag...",,,s/i,mx/t/2010/82012,2026,,,23 de febrero,karina aurora gonzalez segura
5,,281654,novartis ag.,mesilato de imatinib (forma ),,02-mayo-2026,"ultimo pago: 28 abril de 2015, siguiente pago:...",,,s/i,mx/t/2011/8447,2026,,,23 de febrero,karina aurora gonzalez segura
6,,281655,novartis ag.,mesilato de imatinib,,08-mayo-2026,"ultimo pago: 28 abril de 2015, siguiente pago:...",,,s/i,mx/t/2011/5444,2026,,,23 de febrero,karina aurora gonzalez segura
7,,303080,centro de ingenieria geneticay biotecnologia; ...,doxorubicina clorhidrato; mesilato de imatinib...,,28-febrero-2027,"ultimo pago: 11 enero de 2017, siguiente pago:...",,,s/i,mx/t/2012/97346,2027,,,23 de febrero,karina aurora gonzalez segura
8,,303194,novartis ag.,mesilato de imatinib,,23-noviembre-2026,"ultimo pago: 07 septiembre de 2012, siguiente ...",,,s/i,mx/t/2012/105676,2026,,,23 de febrero,karina aurora gonzalez segura
9,,306129,"bioniche life sciences, inc.",mesilato de imatinib (forma ),,03-junio-2025,"ultimo pago: 14 diciembre de 2012, siguiente p...",,,s/i,mx/t/2013/27058,2025,,,23 de febrero,karina aurora gonzalez segura
10,,308263,novartis ag.,mesilato de imatinib,,20-septiembre-2027,"ultimo pago: 27 marzo de 2013, siguiente pago:...",,,s/i,mx/t/2013/46564,2027,,,23 de febrero,karina aurora gonzalez segura
